In [ ]:
import pandas as pd
import requests
import json

import os
from dotenv import load_dotenv
load_dotenv()
DATABASE_CONNECTION_URI = os.environ["DB_URL"]
API_KEY = os.environ["API_KEY"]
DOMAIN = os.environ["DOMAIN"]
PASSWORD = os.environ["PASSWORD"]

# create a connection to the database
from sqlalchemy import create_engine
engine = create_engine(DATABASE_CONNECTION_URI)

# DB Schema Handling
api_schema = json.load(open("api_schema_companies_nov_2023.json"))["API"]
freshdesk_schema = api_schema["Freshdesk"]

class FreshdeskAPI:
    def __init__(self):
        # IDs
        self.api_key = API_KEY
        self.domain = DOMAIN
        self.password = PASSWORD

    def raw_export_companies(self, properties):
        q_results = 101
        page = 1
        results = []
        while q_results >= 100:
            api_response = requests.get(
                "https://" + self.domain + ".freshdesk.com/api/v2/companies?per_page=100&page=" + str(
                    page),
                auth=(self.api_key, self.password))
            api_response = json.loads(api_response.content)
            # Iterate through the list and print each element
            # for item in api_response:
            #     print(item)
            q_results = len(api_response)
            page += 1
            for index, d in enumerate(api_response):
                for prop in list(d):
                    if type(d[prop]) is dict:
                        for item in list(d[prop]):
                            api_response[index].update({prop + "_" + item: api_response[index][prop][item]})
                        api_response[index].pop(prop)
            results.extend(api_response)
            print("Freshdesk Company Export has gathered " + str(len(results)) + " Companies")
        target_results = []
        for index, d in enumerate(results):
            d = {key: value for key, value in results[index].items() if key in properties}
            target_results.append(d)
        df = pd.DataFrame(target_results)
        df['id'] = df['id'].astype(str)

        # inserta la data resultado de la extraccion en una tabla de la DB postgre
        df.to_sql('freshdesk_companies', con=engine, if_exists='replace', index=False)
    

In [ ]:
properties = {
    'id': 1,
    'name': 1,
    #'account_tier': 1,
    'created_at': 1,
}

instance = FreshdeskAPI()

instance.raw_export_companies(properties)

